# This notebook gets soil data thanks to SoilGrids (https://soilgrids.org/) through API requests

Imports and config

In [18]:
import os
os.environ["OGR_GEOJSON_MAX_OBJ_SIZE"] = "0"

import geopandas as gpd
import requests
import pandas as pd
import json
import random
import hashlib
import shapely
from shapely.geometry import Point
from concurrent.futures import ThreadPoolExecutor, as_completed
from tenacity import retry, stop_after_attempt, wait_fixed
from tqdm.notebook import tqdm  

# Config
CACHE_DIR = "soil_cache"
OUTPUT_CSV = "aoc_soilgrids.csv"
MAX_WORKERS = 8
os.makedirs(CACHE_DIR, exist_ok=True)


In [ ]:
# Sample random points inside a polygon
def sample_points_in_polygon(polygon, num_points=3):
    points = []
    minx, miny, maxx, maxy = polygon.bounds
    attempts = 0
    while len(points) < num_points and attempts < num_points * 10:
        pt = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(pt):
            points.append(pt)
        attempts += 1
    return points

# Retry-enabled SoilGrids API query (minimal data)
@retry(stop=stop_after_attempt(3), wait=wait_fixed(2))
def get_soilgrids_data(
    lat, lon,
):
    base = "https://rest.isric.org/soilgrids/v2.0/properties/query"

    # build the repeated query‑string keys
    params = [("lon", lon), ("lat", lat), ("depth", depth)]
    params += [("property", p) for p in properties]

    r = requests.get(base, params=params, timeout=30)
    r.raise_for_status()
    return r.json()


# Local file cache for SoilGrids API
def get_cached_soilgrids_data(lat, lon):
    key = hashlib.md5(f"{lat:.5f}_{lon:.5f}".encode()).hexdigest()
    cache_path = os.path.join(CACHE_DIR, f"{key}.json")
    if os.path.exists(cache_path):
        with open(cache_path, 'r') as f:
            return json.load(f)
    try:
        data = get_soilgrids_data(lat, lon)
        with open(cache_path, 'w') as f:
            json.dump(data, f)
        return data
    except Exception:
        return None

# Extract topsoil values from response
def extract_topsoil_values(
    data: dict,
    properties=("phh2o", "ocd", "clay", "sand", "silt", "bdod"),
    wanted_depth="0-5cm",
):
    """
    Returns {prop: value or None}.
    """
    out = {p: None for p in properties}
    if not data:
        return out

    if "properties" in data and isinstance(data["properties"], dict) and "layers" in data["properties"]:
        for layer in data["properties"]["layers"]:
            name = layer.get("name")
            if name in properties:
                depth_obj = next(
                    (d for d in layer.get("depths", []) if d.get("label") == wanted_depth),
                    None,
                )
                if depth_obj:
                    val    = depth_obj["values"].get("mean")
                    d_fac  = layer.get("unit_measure", {}).get("d_factor", 1)
                    out[name] = val / d_fac if val is not None else None
        return out

    # Unknown layout
    return out


In [22]:
# Load the AOC GeoJSON (ensure the file is uploaded)
aoc_gdf = gpd.read_file("aoc_polygons.geojson").to_crs(epsg=4326)
aoc_gdf.head()


,app,type_prod,categorie,geometry
0,Ajaccio,Vins,Vin tranquille,"MULTIPOLYGON (((8.60355 42.14345, 8.60212 42.1..."
1,Aloxe-Corton,Vins,Vin tranquille,"MULTIPOLYGON (((4.85911 47.05603, 4.8558 47.05..."
2,Alsace grand cru Altenberg de Bergbieten,Vins,"Vin de sélection de grains nobles, Vin de vend...","POLYGON ((7.45994 48.58269, 7.45994 48.58219, ..."
3,Alsace grand cru Altenberg de Bergheim,Vins,"Vin de sélection de grains nobles, Vin de vend...","MULTIPOLYGON (((7.35264 48.2073, 7.34906 48.20..."
4,Alsace grand cru Altenberg de Wolxheim,Vins,"Vin de sélection de grains nobles, Vin de vend...","POLYGON ((7.5107 48.57401, 7.51041 48.57069, 7..."


In [23]:
# Resume partial work if available
existing = pd.read_csv(OUTPUT_CSV) if os.path.exists(OUTPUT_CSV) else pd.DataFrame()
done_aocs = set(existing["AOC"]) if "AOC" in existing.columns else set()

# Adjust the number of AOCs processed for testing
aoc_to_process = aoc_gdf[~aoc_gdf["app"].isin(done_aocs)] #.head(10)
print(f"Resuming. Already processed: {len(done_aocs)}, Remaining: {len(aoc_to_process)}")


Resuming. Already processed: 0, Remaining: 357


In [24]:
# Process one AOC polygon: sample points, get soil data, average
def process_aoc(row):
    try:
        name = row["app"]
        polygon = row["geometry"]
        points = sample_points_in_polygon(polygon, num_points=5)

        point_values = []
        for pt in points:
            data = get_cached_soilgrids_data(pt.y, pt.x)
            values = extract_topsoil_values(data)
            point_values.append(values)

        df = pd.DataFrame(point_values)
        mean_values = df.mean(numeric_only=True).to_dict()
        mean_values["AOC"] = name
        return mean_values
    except Exception as e:
        return {"AOC": row.get("app", "Unknown"), "error": str(e)}


In [25]:
# Run AOC sampling in parallel
results = []

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = [executor.submit(process_aoc, row) for idx, row in aoc_to_process.iterrows()]
    for f in tqdm(as_completed(futures), total=len(futures), desc="Processing AOCs"):
        results.append(f.result())


Processing AOCs:   0%|          | 0/357 [00:00<?, ?it/s]

In [26]:
# Append new results and save
new_df = pd.DataFrame(results)
final_df = pd.concat([existing, new_df], ignore_index=True)
final_df.to_csv(OUTPUT_CSV, index=False)
final_df.head()


,phh2o,ocd,clay,sand,silt,bdod,AOC
0,6.850000,45.900000,29.875000,24.225000,45.900000,1.302500,Alsace grand cru Eichberg
1,6.650000,43.450000,29.100000,21.125000,49.775000,1.275000,Alsace grand cru Altenberg de Bergbieten
2,6.700000,51.166667,31.766667,17.500000,50.766667,1.316667,Aloxe-Corton
3,6.733333,44.333333,21.233333,46.500000,32.266667,1.183333,Ajaccio
4,6.733333,50.900000,29.533333,26.233333,44.233333,1.290000,Alsace grand cru Brand
